In [1]:
from gensim.models import KeyedVectors

model_path = "GoogleNews-vectors-negative300.bin"
model = KeyedVectors.load_word2vec_format(model_path, binary=True)

In [55]:
import pandas as pd
import numpy as np
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import contractions
from nltk.stem import WordNetLemmatizer

### Q1

In [15]:
def predict_message_class(model, w2v_model, message):
    total = 0
    temp = 0
    for word in message:
        if word in w2v_model.key_to_index:
            total += 1
            temp += w2v_model[word]
    if total > 0:
        message = [temp / total]
    else:
        message = [[0]*300]
    prediction = model.predict(message)
    return "ham" if prediction==1 else "spam"

In [48]:
stop_words = stopwords.words('english')
lines = []
with open("spamhamdata.csv") as file:
    lines = file.readlines()
lines
labels = []
classes = []
messages = []
vectors = []
for line in lines:
    label, message = line.split("\t")
    message.replace("\n", "")
    labels.append(label)
    classes.append(1 if label=="ham" else 0)
    message = message.lower()
    messages.append(message)
    message_word = message.split()
    result_words  = [word for word in message_word if word not in stop_words]
    message = ' '.join(result_words)
    vectors.append(WhitespaceTokenizer().tokenize(message))
for i in range(len(vectors)):
    total = 0
    temp = 0
    for word in vectors[i]:
        if word in model.key_to_index:
            total += 1
            temp += model[word]
    if total != 0:
        vectors[i] = temp / total
    else:
        vectors[i] = [0]*300
    X_train, X_test, y_train, y_test = train_test_split(vectors, classes, train_size=0.8, random_state=95)
    LR = LogisticRegression()
    LR.fit(X_train, y_train)
    y_pred = LR.predict(X_test)
    print(accuracy_score(y_test, y_pred))
# for i in range(len(messages)):
#     print(messages[i])
#     print(f"{labels[i]}: ", end="")
#     print(predict_message_class(LR, model, messages[i]))

0.9596412556053812


### Q2

In [105]:
def predict_tweet_sentiment(model, w2v_model, tweet):
    total = 0
    temp = 0
    for word in tweets[i].lower().split():
        word.replace("?", "")
        word.replace("!", "")
        word.replace(".", "")
        word.replace(",", "")
        word.replace(";", "")
        word.replace(":", "")
        word.replace("\"", "")
        word.replace("'", "")
        if word[0] == "@" or "http" in word or "com" in word or "//" in word:
            continue
        word_expanded = contractions.fix(word)
        for new_word in word_expanded:
            new_word = lemmatizer.lemmatize(new_word)
            if word in w2v_model.key_to_index:
                total += 1
                temp += w2v_model[word]
    if total != 0:
        tweet = [temp / total]
    else:
        tweet = [[0]*300]
    prediction = model.predict(tweet)
    return "positive" if prediction==1 else "neutral" if prediction==0 else "negative"

In [59]:
lemmatizer = WordNetLemmatizer()
tweets_df = pd.read_csv("Tweets.csv")
sentiments = tweets_df["airline_sentiment"]
tweets = tweets_df["text"]
classes = []
vectors = []
for i in range(len(tweets)):
    temp = 0
    total = 0
    sentiment = sentiments[i]
    for word in tweets[i].lower().split():
        word.replace("?", "")
        word.replace("!", "")
        word.replace(".", "")
        word.replace(",", "")
        word.replace(";", "")
        word.replace(":", "")
        word.replace("\"", "")
        word.replace("'", "")
        if word[0] == "@" or "http" in word or "com" in word or "//" in word:
            continue
        word_expanded = contractions.fix(word)
        for new_word in word_expanded:
            new_word = lemmatizer.lemmatize(new_word)
            if word in model.key_to_index:
                total += 1
                temp += model[word]
    if total != 0:
        vectors.append(temp / total)
    else:
        vectors.append([0]*300)
    classes.append(1 if sentiment=="positive" else 0 if sentiment=="neutral" else -1)

In [107]:
X_train, X_test, y_train, y_test = train_test_split(vectors, classes, train_size=0.8, random_state=95)
LR = LogisticRegression()
LR.fit(X_train, y_train)
y_pred = LR.predict(X_test)
print(accuracy_score(y_test, y_pred))
# for i in range(len(tweets)):
#     print(tweets[i])
#     print(f"{sentiments[i]}: ", end="")
#     print(predict_tweet_sentiment(LR, model, tweets[i]))

0.7745901639344263
